<center>
    <img src="https://alex-demo-tps-cos-standard-gdi.s3.us-south.cloud-object-storage.appdomain.cloud/b-tps-labs-01de03_2.png" width="640" alt="TPS Header"  />
</center>


# Coursera Capstone - Professional Applied Data Science
## by Alex A.

Tiempo estimado necesario **desconocido** minutos

## Objetive

To show assignment results on Capstone:
- Hello world
- Segmenting and Clustering Neighborhoods in Toronto
- ...

In [51]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

In [52]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

1. Start by creating a new Notebook for this assignment. <br>
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [53]:
import requests
from bs4 import BeautifulSoup

wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(wiki).text

s = requests.Session()
response = s.get(wiki,timeout=10)
response

<Response [200]>

This code is optional 
<!--
#soup = BeautifulSoup(website_url,'html')
soup = BeautifulSoup(website_url,'lxml')
print(soup)
-->

This code is optional 
<!--
soup = BeautifulSoup(website_url,'html.parser')
print(soup)
-->

In [109]:
soup = BeautifulSoup(website_url, 'lxml')
#print(soup)
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [152]:
my_table  = soup.find_all('table')
#print(my_table)

In [111]:
# Set table to contentTable
contentTable  = soup.find('table', { "class" : "wikitable sortable"}) # Use dictionary to pass key : value pair
# Set header to h_rows
h_rows  = contentTable.find_all('th')
for row in h_rows:
    print(row.get_text())

Postal Code

Borough

Neighbourhood



In [151]:
#Set the 'header'
header = [th.text.rstrip() for th in contentTable.find_all('th')]
print(header)
#print('-------')
#print(len(header))

['Postal Code', 'Borough', 'Neighbourhood']


In [139]:
#Set the 'body' (data rows) -- notice that this case has only 3 columns
c1 = []
c2 = []
c3 = []
for td in contentTable.findAll('tr'):
    cells = td.findAll('td')
    if len(cells)==3:
        c1.append(cells[0].text.rstrip())
        c2.append(cells[1].text.rstrip())
        c3.append(cells[2].text.rstrip())

In [149]:
# Set all in a dict called: 'd'
d = dict([(x,0) for x in header])
d['Postal Code'] = c1
d['Borough'] = c2
d['Neighbourhood'] = c3

In [191]:
# Now, create the dataframe from recent dict 
df = pd.DataFrame(d)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [192]:
# size before some clean up
df.shape

(180, 3)

3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [202]:
# Looking for Borough equal to 'Not assigned'
Not_assigned = df[(df['Borough'] == 'Not assigned')].index

In [203]:
# Drop 'Not assigned' and re-assing to dataframe
df = df.drop(Not_assigned)

In [206]:
df.head().reset_index(drop = True)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [212]:
df.shape

(103, 3)

## Using - IBM Cloud Pak for Data 
Very easy load file and load into dataframe

In [213]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_79a5e0e1912648b29b2e33bbc86945f0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='00b6VBtXQz_k4vBc3yfb-hGgSoZp9gIQFf6leNLV-k9C',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_79a5e0e1912648b29b2e33bbc86945f0.get_object(Bucket='courseraexercises-donotdelete-pr-inmwfwfxo8af8e',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )




In [214]:
df_ll = pd.read_csv(body)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [215]:
dfc = df.merge(df_ll, left_on='Postal Code', right_on='Postal Code')
dfc.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [216]:
dfc.shape

(103, 5)